#1. Introducción y Objetivo de la Preparación

En esta etapa del proyecto seguimos la metodología CRISP-DM, ubicándonos en la fase de Preparación de los Datos. El objetivo principal de este notebook es transformar y construir un dataset que esté listo para su uso en la fase de modelado, garantizando que los datos sean relevantes, consistentes y estén estructurados de manera óptima para entrenar modelos de machine learning.

Durante la fase previa (Comprensión de los Datos), identificamos múltiples fuentes de información relevantes: train.csv, test.csv, historical_transactions.csv, new_merchant_transactions.csv y merchants.csv. También detectamos distintos problemas de calidad de datos, como valores faltantes, outliers, y estructuras que requerían integrarse.

Por tanto, en esta notebook ejecutaremos las tareas necesarias para:

- Seleccionar atributos pertinentes para el modelado desde las distintas fuentes.

- Integrar adecuadamente las tablas relacionales (joins y agregaciones).

- Limpiar los datos detectados como problemáticos.

- Generar nuevas variables a partir del comportamiento transaccional de los usuarios.

- Transformar el dataset final a un formato y estructura compatibles con algoritmos de modelado.

El resultado final será un dataset preparado y curado que contenga una representación sintética de cada tarjeta (card_id), con variables numéricas y categóricas que permitan predecir de forma eficaz el loyalty score.

##Resumen de decisiones tomadas durante la fase de comprensión de los datos (por ejemplo, limpieza, selección de atributos).
Durante la fase de comprensión de los datos se analizaron las distintas fuentes de información provistas por Elo: datos de clientes (train.csv), transacciones históricas (historical_transactions.csv), transacciones recientes (new_merchant_transactions.csv) y características de los comercios (merchants.csv). Este análisis permitió identificar patrones clave, valores ausentes, relaciones entre variables y posibles outliers.

En base a los hallazgos anteriores, la fase de preparación de datos tiene como objetivo transformar estos datos crudos en un conjunto estructurado, limpio y listo para alimentar modelos de aprendizaje automático supervisado. Las principales decisiones tomadas que guían esta etapa son:

- El score de lealtad (target) es una variable continua y será predicho mediante un modelo de regresión.

- Se seleccionaron variables relevantes de cada archivo, considerando tanto su correlación potencial con la variable objetivo como su valor semántico en el contexto del negocio.

- Se detectaron variables con valores faltantes y outliers, que requerirán tratamiento adecuado.

- Se observó que muchas variables relevantes provienen de las transacciones, por lo que se realizarán agregaciones por cliente (card_id) para construir una matriz de datos con un registro por cliente.

- Las variables temporales tienen potencial para capturar estacionalidad, antigüedad y frecuencia, por lo que serán transformadas y derivadas.

- Se aplicará codificación a variables categóricas y se descartarán aquellas con baja varianza o sin valor informativo.

#2. Selección de Datos

##Variables del archivo train.csv
- card_id: Identificador único del cliente. Será la clave principal para unir los datasets.

- first_active_month: Se utilizará para derivar la antigüedad del cliente.

- feature_1, feature_2, feature_3: Variables categóricas internas provistas por Elo. Si bien su significado exacto no está documentado, muestran cierta variabilidad entre clientes y potencialmente capturan preferencias o segmentaciones internas.

Estas variables representan el punto de partida del cliente en la plataforma y pueden capturar segmentaciones internas o ciclos de vida que impactan en su lealtad futura.

##Variables derivadas del archivo historical_transactions.csv (agregadas por card_id)
- purchase_amount: Se agregará como total, promedio, desviación estándar y percentiles. Mide el volumen histórico de gasto.

- installments: Agregado como promedio, mediana y proporción de pagos a cuotas. Refleja el tipo de consumo (efectivo vs crédito).

- purchase_date: Se extraerán variables de tiempo como antigüedad de la última compra, dispersión temporal entre compras y estacionalidad.

- month_lag: Permite agrupar transacciones por su antigüedad relativa.

- authorized_flag: Porcentaje de transacciones aprobadas. Mide fiabilidad del cliente.

- category_1, category_2, category_3: Codificadas como proporciones o frecuencias. Capturan tipo de promoción o categoría del gasto.

- merchant_category_id, state_id, city_id: Se usará diversidad (número único de categorías o ubicaciones visitadas). Refleja amplitud geográfica y de intereses.

Estas variables son fundamentales para entender el comportamiento histórico del cliente, sus patrones de gasto, regularidad y preferencias. Permiten inferir su valor y fidelidad previa al sistema de Elo.

##Variables derivadas del archivo new_merchant_transactions.csv
- Misma estructura y tratamiento que historical_transactions, pero se analizarán como indicadores de comportamiento reciente (últimos comercios visitados).

Permiten medir el comportamiento del cliente frente a nuevos comercios, lo que es clave para inferir respuesta a campañas, apertura al cambio y potencial de exploración comercial. Clientes activos con nuevos comercios pueden tener mayor probabilidad de lealtad futura.

##Variables del archivo merchants.csv (vinculadas vía merchant_id)
- avg_sales_lag3, avg_sales_lag6, etc.: Indicadores del dinamismo del comercio.

- most_recent_sales_range, most_recent_purchases_range: Codifican la intensidad comercial más reciente. Se codificarán ordinalmente.

- merchant_category_id, category_1, category_4: Se cruzarán con las transacciones para crear representaciones de afinidad del cliente con ciertas categorías o tipos de comercio.

El tipo de comercio influye en el patrón de compra del cliente. Al cruzarlo con los hábitos del cliente, se pueden obtener variables que midan la compatibilidad cliente–comercio.

##Variables descartadas o no priorizadas
- subsector_id, category_name: Alta cardinalidad, bajo control semántico. Serán descartadas a menos que pruebas futuras muestren valor predictivo.

- Variables como merchant_id y merchant_group_id no se usarán directamente, pero servirán como llaves de agregación.

- Transacciones no autorizadas: Si representan un porcentaje muy pequeño (<5%), pueden ser descartadas por ruido.

- Variables con más del 90% de valores nulos o con baja varianza.

##Enfoque adicional orientado al producto
- Clientes con gran diversidad de categorías y alta frecuencia reciente podrían representar perfiles altamente leales o promotores. Este tipo de hipótesis será validada en el modelado.

- Se prestará atención a variables que permitan explicar no solo el monto gastado, sino el nivel de compromiso con la plataforma, como uso recurrente, exploración de nuevos comercios y respuesta a promociones.

In [1]:
# Carga de Datos
import pandas as pd

# Cargar archivos (ajusta las rutas según tu entorno)
train = pd.read_csv("src/train.csv", parse_dates=["first_active_month"])
historical = pd.read_csv("src/historical_transactions.csv", parse_dates=["purchase_date"])
new_merchant = pd.read_csv("src/new_merchant_transactions.csv", parse_dates=["purchase_date"])
merchants = pd.read_csv("src/merchants.csv")

In [2]:
# Variables seleccionadas del dataset train
train_selected = train[["card_id", "first_active_month", "feature_1", "feature_2", "feature_3", "target"]]

# Variables seleccionadas del dataset historical_transactions
historical_selected = historical[
    ["card_id", "purchase_date", "purchase_amount", "installments", "month_lag",
     "category_1", "category_2", "category_3", "merchant_id", "state_id", "city_id"]
]

# Variables seleccionadas del dataset new_merchant_transactions
new_merchant_selected = new_merchant[
    ["card_id", "purchase_date", "purchase_amount", "installments", "month_lag",
     "category_1", "category_2", "category_3", "merchant_id", "state_id", "city_id"]
]

# Variables seleccionadas del dataset merchants
merchants_selected = merchants[
    ["merchant_id", "merchant_group_id", "category_1", "category_2", "avg_sales_lag3",
     "avg_sales_lag6", "avg_sales_lag12", "avg_purchases_lag3", "avg_purchases_lag6", 
     "avg_purchases_lag12", "most_recent_sales_range", "most_recent_purchases_range",
     "category_4", "city_id", "state_id"]
]


In [3]:
# Guardar los DataFrames seleccionados en archivos CSV
train_selected.to_csv("src/train_selected.csv", index=False)
historical_selected.to_csv("src/historical_selected.csv", index=False)
new_merchant_selected.to_csv("src/new_merchant_selected.csv", index=False)
merchants_selected.to_csv("src/merchants_selected.csv", index=False)


#3. Limpieza de los Datos

##Imputación de valores faltantes.

##Detección y tratamiento de outliers.

##Corrección de valores erróneos, inconsistentes o duplicados.

##Normalización de formatos (ej: fechas, strings, booleans).

#4. Integración de Datos

##Uniones entre tablas (por ejemplo: train + transacciones + merchants).

##Definición del tipo de join (left, inner, etc.) y justificación.

##Manejo de duplicados y verificación de cardinalidades.

##Agregaciones por tarjeta, tiempo o categoría (u otras variables).

#5. Construcción de Variables

##Generación de nuevas variables basadas en las existentes (feature engineering).

In [ ]:
# Ejemplos: total de compras, frecuencia de uso, tipo de comercio más visitado.

##Cálculo de estadísticas agregadas por cliente, comerciante o tiempo.

In [ ]:
# u otras agregaciones

##Justificación de cada nueva variable.

#6. Formateo y Transformación

##Escalado de variables numéricas (min-max, z-score, log, etc.).

In [ ]:
# Se pueden probar varios escalados para testear luego con el modelado.
# O investigar formas de validar la calidad del escalado (graficos u otros metodos)

##Codificación de variables categóricas (label encoding, one-hot, etc.).

##Conversión de fechas a características útiles (mes, día, antigüedad, etc.).

#7. Filtrado Final del Dataset

##Eliminación de columnas redundantes o intermedias.

##Confirmación de que no existen valores nulos o inconsistencias.

##Revisión de balance de clases si aplica.

#8. Almacenamiento del Dataset Final

##Exportación del dataset limpio en formato .csv o .parquet.

##Guardado de versiones intermedias si es necesario.

##Comentarios sobre estructura y uso futuro del dataset.

#9. Validación del Dataset para Modelado

##Comprobación de dimensiones, tipos de datos, consistencia.

##Revisión rápida de la distribución de la variable target.

##Verificación de integridad después de joins y transformaciones.

#10. Conclusiones y Próximos Pasos

##Resumen de pasos realizados.
###1. adsad
-
-
-

###2. adsad
-
-
-

###3. adsad
-
-
-


##Observaciones relevantes para el modelado.
-
-
-


##Criterios pendientes o decisiones abiertas
- (por ejemplo: pruebas de codificaciones)
-
-